In [10]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from libpysal.weights import DistanceBand
import esda
from splot.esda import moran_scatterplot
from matplotlib_scalebar.scalebar import ScaleBar

# Load the GeoDataFrame
cat = gpd.read_file("G:/My Drive/INVESTIGACION/PAPERS/ELABORACION/Modelo_SAR/DATA/df_catchments_kmeans.gpkg")

# Create a DistanceBand weights matrix
w_dist20km = DistanceBand.from_dataframe(cat, 20000, binary=False)

# List of variables and their respective x-axis labels
variables = [
    ('area', 'Área normalizada'),
    ('hypso_inte', 'Hipso normalizada'),
    ('slope_mean', 'Pendiente normalizada'),
    ('rainfallAnnual_mean', 'Lluvia normalizada')
]

# Loop over each variable, calculate the Moran's I and plot
for variable, xlabel in variables:
    # Calculate local Moran's I
    lisa = esda.Moran_Local(cat[variable], w_dist20km)

    # Create subplots with consistent sizes
    f, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 7))  # Adjusted figsize for better uniformity
    axs = ax.flatten()

    # Local Moran's I spatial plot
    q_labels = ['Q1', 'Q2', 'Q3', 'Q4']
    labels1 = [q_labels[i - 1] for i in lisa.q]
    hmap = ListedColormap(['red', 'lightblue', 'blue', '#FAD7A0'])
    cat.assign(cl=labels1).plot(column='cl', categorical=True, k=2, cmap=hmap, linewidth=0.1, ax=ax[0], edgecolor='white', legend=True)

    # Significance plot overlay
    sig = 1 * (lisa.p_sim < 0.05)
    hmap = ListedColormap(['grey', 'black'])
    labels2 = ['non-sig.', 'significant']
    labels2 = [labels2[i] for i in sig]
    cat.assign(cl=labels2).plot(column='cl', categorical=True, k=2, cmap=hmap, linewidth=0.1, ax=ax[0], edgecolor='white', alpha=0.20)
    ax[0].set_aspect('equal')
    ax[0].set_axis_off()

    # Moran scatterplot
    moran_scatterplot(lisa, p=0.05, ax=ax[1])
    # Calculate the global Moran's I if you want to display it
    mi = esda.Moran(cat[variable], w_dist20km)  # Calculate the global Moran's I
    ax[1].text(-2, 1.5, f'MI={round(mi.I, 2)}', fontsize=15)  # Increased font size by 50%
    ax[1].set_xlabel(xlabel, fontsize=15)  # Increased font size by 50%
    ax[1].set_ylabel("Autoregresión espacial", fontsize=15)  # Increased font size by 50%
    ax[1].set_title('')

    # Adjust the aspect ratio and limits of ax[1] to maintain consistent size
    ax[1].set_aspect('auto')  # You can set this to 'equal' or 'auto' based on your preference
    ax[1].set_xlim([-2.5, 2.5])  # Set consistent limits (adjust as needed)
    ax[1].set_ylim([-2.5, 2.5])  # Adjust as needed for consistent sizing

    # Add a north arrow (assuming the function is defined elsewhere)
    add_north_arrow(ax[0], scale=.75, xlim_pos=.2, ylim_pos=.965, color='#000', text_scaler=4, text_yT=-1.25)

    # Add a scale bar
    scalebar = ScaleBar(1, "m", location="lower right", scale_loc="top", length_fraction=0.25)
    ax[0].add_artist(scalebar)

    # Save the figure
    output_path = f"G:/My Drive/INVESTIGACION/PAPERS/ELABORACION/Modelo_SAR/FIGURAS/{variable}_moran.png"
    plt.savefig(output_path, dpi=500)
    plt.close()  # Close the plot to avoid overlap in the next iteration



C:\Users\edier\miniconda3\Lib\site-packages\scipy\sparse\_data.py:119: RuntimeWarning: divide by zero encountered in reciprocal
  return self._with_data(data ** n)
C:\Users\edier\miniconda3\Lib\site-packages\libpysal\weights\weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  warnings.warn(message)
